In [ ]:
# Basically, this takes a folder of 2D-raster (NDVI, NDWI,NBR or any other ), stacks them, and then returns new rasters

# containig pixel-by-pixel trend analyses along the 3rd axis (r-value, slope, etc.)
#*-**********************************************************************************
#***********Make sure the bands are in BSQ not in BIP format***************************
#**********************************************************************************

In [1]:
#import libraries
import numpy as np
from gdalconst import *
from osgeo import gdal
import numpy as np
from gdalconst import *
import  ogr, sys, osr,glob, os
gdal.UseExceptions()
import scipy
from scipy.stats import norm


In [4]:
#####################################################
################## FUNCTIONS ########################
#####################################################
# Function that reads an entire folder of rasters as arrays and stores them in a list,
# default raster input format is GeoTiff
# printOut is only used to debug the code

def tiffToarray(inFol, printOut = False, inFormat = "tif"):

    # Use the first raster as a blueprint for the raster size of all rasters

    listdir = glob.glob("*.tif")
    for allRasters in listdir:
        if allRasters[-3:] == "tif":
            firstRasStr = os.path.join(inFol , allRasters)
            print("1st raster:", firstRasStr)
            break


    driver = gdal.GetDriverByName('GTiff')
    firstRasGDAL = gdal.Open(firstRasStr, GA_ReadOnly)
    if firstRasGDAL is None:
        print("dataset not existant")
        sys.exit(-1)
    cols = firstRasGDAL.RasterXSize
    rows = firstRasGDAL.RasterYSize
    
    # initialise list that will store all arrays to be returned in the end
    finList = [] 

    for files in listdir:
        if files[-3:] == inFormat:
            if printOut:
                print(files)

            fileIn = os.path.join(inFol , files)
            dataset = gdal.Open(fileIn, GA_ReadOnly)
            print("fileIn:", fileIn)
            array = dataset.ReadAsArray(0, 0, cols, rows)
            finList.append(array)
            #print("shapeOfImg: ",cols,rows)
    return finList

def linReg(inList):

    #equally spaced time steps by length of inList
    timeList = np.asarray(list(range(len(inList))))
    tl = np.asarray(inList)
    stepLen = len(inList)
    print("leng of inlist:", timeList.shape,"tl:", tl.shape, "steplen:",stepLen)
        
    #stack input arrays to make a 3D array    
    dstack = np.dstack((inList))
    
    #dstack1D = 1st pixel reads values for all images. then 2 nd pixel in a row is read for all    
    dstack1D = dstack.reshape(-1)
   
    dstackList = [dstack1D[i:i+stepLen] for i in range(0, len(dstack1D), stepLen)]
    
    #****THIS IS FIRST 18 PIXELS THAT REPRESENT 18 ARRAYS WITH 6 PIX IN AN ARRAY FROM EACH BAND
    #print("dstackList:", dstackList[:18])
    
    #initialise empty arrays to be filled by output values, arrays are 1D
    slopeAr,intcptAr,rvalAr,pvalAr,stderrAr,mkPAr, mkHAr,mkZAr = [np.zeros(inList[0].reshape(-1).shape) for _ in range(8)]
    
     # Use map() to iterate over each pixels timestep values for linear reg and Mann.Kendall    
    #59*174 = 10384 pixelse in total in one band. for each of these we have 6 values we do linear regression
    #outListReg produces a list [slope,interc,r,p,stderr, mkPAr, mkHAr,mkZAr]
    outListReg = list(map( (lambda x: scipy.stats.linregress(timeList, x)) , dstackList))    
    
    #this is MK_test to find if there is a trend. input is array of 6 values at each pixel.
    outListMK = list(map( (lambda x: mk_test(x)) , dstackList))
    #MK test produces( trend,hypot,p,z values). however here mk_test returns 2 values "h and p". outListMK ret
    #for each pixel from 0 to total pixels in 2D
    print("MK len:",len(outListMK) ,  outListMK[:12])
    for k in range(len(outListReg)):  
        #all values in output are from Linear REgressiona dn mkPR comes from MK test.
        slopeAr[k] = outListReg[k][0]
        intcptAr[k] = outListReg[k][1]
        rvalAr[k] = outListReg[k][2]
        pvalAr[k] = outListReg[k][3]
        stderrAr[k] = outListReg[k][4]
        mkPAr[k] = outListMK[k][1]
        mkHAr[k] = outListMK[k][0]
        mkZAr[k] = outListMK[k][2]
    outShape = inList[0].shape
    
    print("TS:", pvalAr.shape,pvalAr.reshape(outShape))
    outTuple = (slopeAr.reshape(outShape), intcptAr.reshape(outShape),  rvalAr.reshape(outShape),  pvalAr.reshape(outShape),stderrAr.reshape(outShape), mkPAr.reshape(outShape), mkHAr.reshape(outShape),mkZAr.reshape(outShape))

    return outTuple

# Mann-Kendall-Test
# Originally from: http://www.ambhas.com/codes/statlib.py

def mk_test(x, alpha = 0.05):
    ''' """
    This function is derived from code originally posted by Sat Kumar Tomer
    (satkumartomer@gmail.com)
    See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm
    The purpose of the Mann-Kendall (MK) test (Mann 1945, Kendall 1975, Gilbert
    1987) is to statistically assess if there is a monotonic upward or downward
    trend of the variable of interest over time. A monotonic upward (downward)
    trend means that the variable consistently increases (decreases) through
    time, but the trend may or may not be linear. The MK test can be used in
    place of a parametric linear regression analysis, which can be used to test
    if the slope of the estimated linear regression line is different from
    zero.'''
    n = len(x)
    # calculate S
    listMa = np.matrix(x)               # convert input List to 1D matrix
    subMa = np.sign(listMa.T - listMa)  # calculate all possible differences in matrix
                                        # with itself and save only sign of difference (-1,0,1)
    s = np.sum( subMa[np.tril_indices(n,-1)] ) # sum lower left triangle of matrix
    # calculate the unique data
    # return_counts=True returns a second array that is equivalent to tp in old version
    unique_x = np.unique(x, return_counts=True)
    g = len(unique_x[0])

    # calculate the var(s)
    if n == g: # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else: # there are some ties in data
        tp = unique_x[1]
        var_s = (n*(n-1)*(2*n+5) + np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s>0:
        z = (s - 1)/np.sqrt(var_s)
    elif s == 0:
            z = 0
    elif s<0:
        z = (s + 1)/np.sqrt(var_s)
    # calculate the p_value
    p = 2*(1-scipy.stats.norm.cdf(abs(z))) # two tail test
    h = abs(z) > scipy.stats.norm.ppf(1-alpha/2)
    return h, p, z

# Function that converts a numpy array to a GeoTIFF and saves it to disk, needs an existing GeoTIFF file as a blueprint
# Changed after the original
# http://gis.stackexchange.com/questions/58517/python-gdal-save-array-as-raster-with-projection-from-other-file
# inTiff  is an exisiting GeoTiff file, the attributes from this file are used to create the output
# array    is the array that will be saved as a tiff
# outFile  is the path and name of the desired output GeoTiff

def array_to_raster(inTiff, array, outFile):
    '''converts array to a tif raster'''
    inDataset = gdal.Open(inTiff, GA_ReadOnly)
    # You need to get those values like you did.
    x_pixels = inDataset.RasterXSize  # number of pixels in x
    y_pixels = inDataset.RasterYSize  # number of pixels in y
    PIXEL_SIZE = inDataset.GetGeoTransform()[1]   # size of the pixel...
    x_min = inDataset.GetGeoTransform()[0]
    y_max = inDataset.GetGeoTransform()[3]   # x_min & y_max are like the "top left" corner.
    wkt_projection = inDataset.GetProjectionRef()
    inDataset = None
    driver = gdal.GetDriverByName('GTiff')
    outDataset = driver.Create( outFile, x_pixels, y_pixels, 1,gdal.GDT_Float32 )
    outDataset.SetGeoTransform((  x_min, PIXEL_SIZE, 0,  y_max,   0,  -PIXEL_SIZE))
    outDataset.SetProjection(wkt_projection)
    outDataset.GetRasterBand(1).WriteArray(array)
    outDataset.FlushCache()  # Write to disk.
    outDataset = None 
    return inTiff

In [5]:

# Example C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\TOA\out_1\WM\out_1
# Take rasters with annual values and return statistical time series trend indices
 # Input folder containing GeoTiffs
os.chdir('C:/_LOCALdata/prj_2019/Burn_severity/PR_imagery/inFol/WM_nbr/' ) 
inFol = os.getcwd()

# Output folder, will contain statistical GeoTiffs after script is run
outFol = 'C:/_LOCALdata/prj_2019/Burn_severity/PR_imagery/inFol/WM_nbr/out/'

#convert GeoTiffs to numpy arrays and stores in a list
inArrays = tiffToarray(inFol) 

 #run linear regression and return arrays of slope, r etc. as a tuple
outTuple = linReg(inArrays)  

## get first raster in list and use it as a blueprint for size etc.
#for allRasters in os.listdir(inFol):
list_dir = glob.glob("*.tif")
for allRasters in list_dir:
    if allRasters[-3:] == "tif":
        firstRasStr = os.path.join(inFol , allRasters)
        break

## define the names of the statistical output rasters
outNames = ("An_slope.tif", "An_intcp.tif", "An_rval.tif", "An_pval.tif", "An_stderr.tif", "An_mkP.tif", "An_mkH.tif","An_mkZ.tif")

## convert the arrays back to rasters and save them to disk
for fname, i in zip(outNames, range(len(outNames))):
    (dir, in_file) = os.path.split(fname)    
    array_to_raster(firstRasStr,outTuple[i],os.path.join(outFol,in_file) )

1st raster: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20130617_20170309_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20130617_20170309_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20130703_20170309_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20130905_20170309_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20130921_20170308_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20140706_20170305_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20140722_20170304_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_nbr\Ind_NBR2_LC08_L1TP_046020_20150709_20170226_cl.tif
fileIn: C:\_LOCALdata\prj_2019\Burn_

RuntimeError: 

In [ ]:
#Import library
from IPython.display import Image
# Load image from local storage
Image(filename = "C:\_LOCALdata\prj_2019\Burn_severity\PR_imagery\inFol\WM_ndvi\extra\out2.png", width = 600, height = 300)